In [1]:
# %run 01_preprocess.ipynb
%run 00_common.ipynb

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


0.27.0


In [2]:
notebook_login()

In [3]:
MODEL_PATTERN='keep_all'
TORGO_TRAIN_TYPE=TorgoTrainType.WORD_KEEP.value

loaded_datasets = {}

for speaker in SPEAKERS:
    file_pattern = f'torgo_xlsr_finetune_{speaker}_{MODEL_PATTERN}_{{split}}.json'

    loaded_datasets[speaker] = load_dataset('json', data_files={
        'train': os.path.join(f'data/{TORGO_TRAIN_TYPE}', file_pattern.format(split='train')),
        'valid': os.path.join(f'data/{TORGO_TRAIN_TYPE}',  file_pattern.format(split='val')),
        'test': os.path.join(f'data/{TORGO_TRAIN_TYPE}',  file_pattern.format(split='test')),
    })

for speaker, dataset in loaded_datasets.items():
    print(f"SPEAKER {speaker} - Train Set Size: {len(dataset['train'])}")
    print(f"SPEAKER {speaker} - Valid Set Size: {len(dataset['valid'])}")
    print(f"SPEAKER {speaker} - Test Set Size: {len(dataset['test'])}")

SPEAKER F01 - Train Set Size: 257
SPEAKER F01 - Valid Set Size: 29
SPEAKER F01 - Test Set Size: 32
SPEAKER F03 - Train Set Size: 251
SPEAKER F03 - Valid Set Size: 28
SPEAKER F03 - Test Set Size: 32
SPEAKER F04 - Train Set Size: 225
SPEAKER F04 - Valid Set Size: 26
SPEAKER F04 - Test Set Size: 28
SPEAKER M01 - Train Set Size: 299
SPEAKER M01 - Valid Set Size: 34
SPEAKER M01 - Test Set Size: 37
SPEAKER M02 - Train Set Size: 420
SPEAKER M02 - Valid Set Size: 47
SPEAKER M02 - Test Set Size: 52
SPEAKER M03 - Train Set Size: 226
SPEAKER M03 - Valid Set Size: 26
SPEAKER M03 - Test Set Size: 28
SPEAKER M04 - Train Set Size: 340
SPEAKER M04 - Valid Set Size: 38
SPEAKER M04 - Test Set Size: 42
SPEAKER M05 - Train Set Size: 348
SPEAKER M05 - Valid Set Size: 39
SPEAKER M05 - Test Set Size: 43


In [4]:
# Initialize the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

In [5]:
def train_model_for_speaker(speaker_id):
    print(speaker_id)
    # Apply preprocess_function to train_data and val_data
    train_data=loaded_datasets[speaker_id]['train']
    val_data=loaded_datasets[speaker_id]['valid']
    train_data = train_data.map(preprocess_function, batched=True)
    val_data = val_data.map(preprocess_function, batched=True)

    # Prepare DataLoader for training and validation
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=16)

    output_dir = f"torgo_spell_correction_{MODEL_PATTERN}_{speaker_id}"
    print(output_dir)

    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=40,
        predict_with_generate=True,
        push_to_hub=False,
        logging_steps=100
    )

    print("Train Dataset:", train_data)
    print("Validation Dataset:", val_data)
    print("Tokenizer:", tokenizer)
    print("Training Arguments:", training_args)

    model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # The Seq2SeqTrainer is created with the defined model, training arguments, datasets, tokenizer, and data_collator
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        tokenizer=tokenizer,
        data_collator=data_collator
    )
    trainer.train()

In [6]:
# train_model_for_speaker('F01')

In [7]:
# train_model_for_speaker('F03')


In [8]:
# train_model_for_speaker('F04')


In [9]:
train_model_for_speaker('M01')

M01


Map:   0%|          | 0/299 [00:00<?, ? examples/s]

torgo_spell_correction_keep_all_M01
Train Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 299
})
Validation Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 34
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,No log,5.967497
2,No log,4.860633
3,No log,3.450475
4,No log,2.779881
5,No log,2.418884
6,5.342700,2.218998
7,5.342700,1.984682
8,5.342700,1.819662
9,5.342700,1.672721
10,5.342700,1.508058


In [10]:
train_model_for_speaker('M02')


M02
torgo_spell_correction_keep_all_M02
Train Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 420
})
Validation Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 47
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,No log,5.064881
2,No log,3.515041
3,No log,2.578561
4,5.408700,2.163491
5,5.408700,1.912642
6,5.408700,1.687630
7,5.408700,1.488878
8,2.356100,1.322984
9,2.356100,1.215303
10,2.356100,1.120166


In [11]:
train_model_for_speaker('M03')


M03


Map:   0%|          | 0/226 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

torgo_spell_correction_keep_all_M03
Train Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 226
})
Validation Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 26
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,No log,5.723696
2,No log,5.041556
3,No log,4.145181
4,No log,3.131651
5,No log,2.682796
6,No log,2.413994
7,5.394800,2.223221
8,5.394800,2.057541
9,5.394800,1.894247
10,5.394800,1.788915


In [12]:
train_model_for_speaker('M04')


M04


Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

torgo_spell_correction_keep_all_M04
Train Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 340
})
Validation Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 38
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,No log,5.322206
2,No log,4.127774
3,No log,2.838535
4,No log,2.379687
5,5.322100,2.082576
6,5.322100,1.877995
7,5.322100,1.679575
8,5.322100,1.501079
9,5.322100,1.383911
10,2.371200,1.238583


In [13]:
train_model_for_speaker('M05')

M05


Map:   0%|          | 0/348 [00:00<?, ? examples/s]

Map:   0%|          | 0/39 [00:00<?, ? examples/s]

torgo_spell_correction_keep_all_M05
Train Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 348
})
Validation Dataset: Dataset({
    features: ['references_phoneme', 'predictions_phoneme', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 39
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,No log,4.769137
2,No log,3.807131
3,No log,2.697385
4,No log,2.271450
5,5.218900,2.076555
6,5.218900,1.851069
7,5.218900,1.691257
8,5.218900,1.555633
9,5.218900,1.411134
10,2.323700,1.350603


In [16]:
from transformers import BartForConditionalGeneration

SPEAKERS = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']
models = {}

for speaker in SPEAKERS:
    model_name = f"torgo_spell_correction_{MODEL_PATTERN}_{speaker}/checkpoint-500"
    models[speaker] = BartForConditionalGeneration.from_pretrained(model_name)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for speaker, model in models.items():
    models[speaker] = model.to(device)
    models[speaker].eval()

In [18]:
def evaluate_speaker(model, tokenizer, speaker_id):
    predictions = []
    references = []
    test_dataset = loaded_datasets[speaker_id]['test']
    test_dataset = test_dataset.map(preprocess_function, batched=True)
    model.eval()

    for example in test_dataset:
        input_text = f"{example['predictions_phoneme']}"
        with torch.no_grad():
            input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
            outputs = model.generate(input_ids=input_ids, max_length=max_length)

        predicted_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

        references.append(example['references_phoneme'])
        predictions.append(predicted_sentence)

    # Verify that the number of predictions and references are the same
    if len(predictions) == len(references):
        print("Number of predictions and references are the same.")
    else:
        print("Mismatch in the number of predictions and references.")

    # Print the number of predictions and references
    print("Number of predictions:", len(predictions))
    print("Number of references:", len(references))
    # print the length of the dataset
    print("Number of rows in dataset:", len(test_dataset))

    # Assuming 'predictions' and 'references' are your sequences
    # Calculate Word Error Rate (WER)
    wer_value = wer(predictions, references)
    wer_percentage = wer_value * 100
    print(f"WER for {speaker_id}: {wer_percentage:.2f}%")

    # Calculate Character Error Rate (CER)
    cer_value = cer(predictions, references)
    cer_percentage = cer_value * 100
    print(f"CER for {speaker_id}: {cer_percentage:.2f}%")

In [19]:
for speaker_id, model in models.items():
    evaluate_speaker(model, tokenizer, speaker_id)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Number of predictions and references are the same.
Number of predictions: 32
Number of references: 32
Number of rows in dataset: 32
WER for F01: 62.26%
CER for F01: 47.73%


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Number of predictions and references are the same.
Number of predictions: 32
Number of references: 32
Number of rows in dataset: 32
WER for F03: 63.64%
CER for F03: 50.00%


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

...

Number of predictions and references are the same.
Number of predictions: 28
Number of references: 28
Number of rows in dataset: 28
WER for M03: 65.22%
CER for M03: 49.49%


Map:   0%|          | 0/42 [00:00<?, ? examples/s]

Number of predictions and references are the same.
Number of predictions: 42
Number of references: 42
Number of rows in dataset: 42
WER for M04: 69.86%
CER for M04: 51.80%


Map:   0%|          | 0/43 [00:00<?, ? examples/s]

Number of predictions and references are the same.
Number of predictions: 43
Number of references: 43
Number of rows in dataset: 43
WER for M05: 61.27%
CER for M05: 49.84%
